In [1]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [2]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [3]:
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.preprocessing import MaxAbsScaler

def peaks(grid):
    x = np.linspace(-2, 2, grid)
    y = np.linspace(-2, 2, grid)
    x, y = np.meshgrid(x, y)
    z = 3*(1-x)**2 * np.exp(-(x**2) - (y+1)**2) - 10*(x/5 - x**3 - y**5) * np.exp(-x**2 - y**2) - 1/3 * np.exp(-(x+1)**2 - y**2)
    return x, y, z

def create_dataframe(grid):
   x, y, z = peaks(grid)
   data = {'X': x.flatten(), 'Y': y.flatten(), 'Z': z.flatten()}
   df = pd.DataFrame(data)
   return df

In [4]:
df_25 = create_dataframe(grid=5)
df_1000 = create_dataframe(grid=32)

In [5]:
def split_df(df):
    # df = df.sample(frac=1).reset_index(drop=True)  # Randomiza as linhas
    _input = np.vstack([df['X'], df['Y']]).T
    _output = np.array(df['Z'])
    return (_input, _output)

In [6]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import levenberg_marquardt as lm
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error
from keras import regularizers
from keras import initializers

class TrainWithSmallDataset:
    def __init__(self, batch_size=1000, store=25):
        self.batch_size = batch_size
        self.betters = []
        self.store = store

    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)


    def split_dataset(self, input, output, sup_input, sup_output):

      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.sup_dataset, self.sup_input, self.sup_output = self.create_dataset(sup_input, sup_output)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)

    def create_model(self, input_size=2, sumary=False):
      regularizer = regularizers.L2(0.2)
      initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

      self.model = tf.keras.Sequential([
          tf.keras.layers.Dense(30,
                                activation='tanh',
                                input_shape=(input_size,),
                                kernel_regularizer=regularizer,
                                kernel_initializer=initializer,
                                ),
          tf.keras.layers.Dense(16,
                                activation='tanh',
                                kernel_regularizer=regularizer,
                                kernel_initializer=initializer,
                                ),
          tf.keras.layers.Dense(1,
                                activation='linear',
                                kernel_regularizer=regularizer,
                                kernel_initializer=initializer,
        )])

      if (sumary == True):
        self.model.summary()

      self.model.compile(
          optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
          loss=tf.keras.losses.MeanSquaredError())

      self.lm_model = lm.ModelWrapper(
          tf.keras.models.clone_model(self.model))

      self.lm_model.compile(
          optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
          loss=lm.MeanSquaredError())

    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.sup_prediction = self.lm_model.predict(self.sup_input)
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()
          sup_pred = self.sup_prediction.flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)
          r2_sup = r2_score(self.sup_output, sup_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)
          mse_sup = mean_squared_error(self.sup_output, sup_pred)

          metrics = {
                          'r2': r2,
                          'r2_sup': r2_sup,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_sup': mse_sup,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics


    def plot_results(self):
      fig = plt.figure(figsize=(18, 6))
      ax0 = fig.add_subplot(1, 3, 1, projection='3d')
      ax2 = fig.add_subplot(1, 3, 2, projection='3d')
      ax1 = fig.add_subplot(1, 3, 3)

      ax0.set_title('Superficie Fit')
      ax0.scatter(self.sup_input[:, 0], self.sup_input[:, 1], self.sup_prediction, c='r', marker='s', label="lm")
      ax0.scatter(self.sup_input[:, 0], self.sup_input[:, 1], self.sup_output, c='b', marker='o', label="reference")
      ax0.legend()

      ax2.set_title('Teste Fit')
      ax2.scatter(self.test_input[:, 0], self.test_input[:, 1], self.test_prediction, c='r', marker='s', label="lm")
      ax2.scatter(self.test_input[:, 0], self.test_input[:, 1], self.test_output, c='b', marker='o', label="reference")
      ax2.legend()

      ax1.plot(self.results.history['loss'], label='Erro Treino')
      ax1.plot(self.results.history['val_loss'], label='Erro validação')
      ax1.set_title('Histórico de Treinamento')
      ax1.set_ylabel('Função de Custo')
      ax1.set_xlabel('Épocas de Treinamento')
      ax1.legend(['Erro Treino', 'Erro validação'])

      return(fig)

2024-01-29 23:32:48.003765: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-29 23:32:48.064900: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-29 23:32:48.064947: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-29 23:32:48.066700: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-29 23:32:48.078058: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-29 23:32:48.079064: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [25]:
from keras import saving
from keras import models
import pickle

class Tester:
  def __init__(self, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input_25, self.output_25 = split_df(df_25)
    self.input_1000, self.output_1000 = split_df(df_1000)
    self.weightsFile 

  def PrepareTraining(self):
    trainer = TrainWithSmallDataset()
    trainer.split_dataset(self.input_25, self.output_25, self.input_1000, self.output_1000)
    return (trainer)

  def Train(self, trainer, epochs=1000):
    trainer.create_model()
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path)
  
  def LoadModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    model.load_weights(path)

  
  def SaveFigs(self, trainer, fileName):
    path = f"../results/{fileName}.png" 
    open(path,'w').close()
    figure = trainer.plot_results()
    figure.savefig(path)
    plt.close(figure)

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)
  
  def LoadDataset(self, fileName, trainer):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'rb') as f:
      _train, _vt, _val, _test = pickle.load(f)
    trainer._train, trainer._vt, trainer._val, trainer._test = _train, _vt, _val, _test

  def LoopWeights(self, sort_by, model_boundarie, trainer, j=0):
    better_model = 0
    for i in range(self.run_times):
      print (f"+++++++++++ Dataset [{j + 1}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] <= model_boundarie): # should be >= to acsending metrics
        fileName = f"model_{j}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.SaveFigs(trainer, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
              

  # brief: for each dataset
  def LoopDataset(self, sort_by, model_boundarie):
    for j in range(self.dataset_run_times):
      trainer = self.PrepareTraining() # split dataset
      self.SaveDataset(trainer, f"dataset_{j}")
      self.LoopWeights(sort_by, model_boundarie, trainer, j)

  def DisplayBetterResults(self, sort_by):
    df = pd.DataFrame.from_dict(self.better_metrics)
    df = df.transpose()
    df = df.sort_values([sort_by])
    display(df)
    excel_path = '../results/metrics.xlsx'
    df.to_excel(excel_path, index=False)
    print(f"DataFrame salvo em {excel_path}")

# Treinando com 25 dados


In [26]:
tester = Tester(run_times=100, dataset_run_times=0)

In [27]:
trainer = tester.PrepareTraining()
tester.LoadDataset("dataset_14", trainer)
tester.LoopWeights(sort_by='mse_sup', model_boundarie=2.5, trainer=trainer)

+++++++++++ Dataset [1] | 1 ++++++++++++++++++


Elapsed time:  0.4506061980000595
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ Dataset [1] | 2 ++++++++++++++++++
Elapsed time:  0.5469138720000046
Stopped at epoch:  7
1/1 [==============================] - 0s 23ms/step
+++++++++++ Dataset [1] | 3 ++++++++++++++++++
Elapsed time:  0.5341333460000897
Stopped at epoch:  8
1/1 [==============================] - 0s 15ms/step
+++++++++++ Dataset [1] | 4 ++++++++++++++++++
Elapsed time:  0.46384320500055765
Stopped at epoch:  7
1/1 [==============================] - 0s 15ms/step
+++++++++++ Dataset [1] | 5 ++++++++++++++++++
Elapsed time:  0.4620744250005373
Stopped at epoch:  6
1/1 [==============================] - 0s 18ms/step
+++++++++++ Dataset [1] | 6 ++++++++++++++++++
Elapsed time:  0.5660074090001217
Stopped at epoch:  8
1/1 [==============================] - 0s 15ms/step
+++++++++++ Dataset [1] | 7 ++++++++++++++++++
Elapsed time:  0.49198746100046264
Stopped at epoch:  6
1/1 [==============

In [30]:
tester.DisplayBetterResults('mse_sup')

KeyError: 'mse_sup'